In [2]:
import scanpy as sc
import pandas as pd

tm_droplet_data = sc.read(
    r'./src/data/tabula_muris/TM_droplet.h5ad',
)
tm_facs_data = sc.read(
    r'./src/data/tabula_muris/TM_facs.h5ad',
)

In [ ]:
tm_droplet_data

In [ ]:
# List all tissue types in tm_droplet_data
tm_droplet_data.obs["tissue"].unique()
# List all tissue types in tm_facs_data
tm_facs_data.obs["tissue"].unique()
# List all cell types in tm_droplet_data
tm_droplet_data.obs["cell_ontology_class"].unique()


In [8]:
# Filter only for cells with valid cell ontology class
tm_droplet_data = tm_droplet_data[
    (~tm_droplet_data.obs.cell_ontology_class.isna())
].copy()
tm_facs_data = tm_facs_data[
    (~tm_facs_data.obs.cell_ontology_class.isna())
].copy()

# Add technology labels
tm_droplet_data.obs["tech"] = "10x"
tm_facs_data.obs["tech"] = "SS2"

In [ ]:
gene_len = pd.read_csv(
    "https://raw.githubusercontent.com/chenlingantelope/HarmonizationSCANVI/master/data/gene_len.txt",
    delimiter=" ",
    header=None,
    index_col=0,
)
gene_len.head()

In [ ]:
# import numpy as np
# from scipy import sparse

# # First, get the gene length data and align it with the FACS data
# gene_len = gene_len.reindex(tm_facs_data.var.index).dropna()
# tm_facs_data = tm_facs_data[:, gene_len.index]

# # Convert to sparse matrix if not already sparse
# if not sparse.issparse(tm_facs_data.X):
#     tm_facs_data.X = sparse.csr_matrix(tm_facs_data.X)

# # Calculate the scaling factor once
# scaling_factor = np.median(gene_len[1].values)

# # Process in chunks to save memory
# chunk_size = 10000  # Adjust based on your available memory
# num_chunks = (tm_facs_data.shape[0] + chunk_size - 1) // chunk_size

# for i in range(num_chunks):
#     start_idx = i * chunk_size
#     end_idx = min((i + 1) * chunk_size, tm_facs_data.shape[0])
    
#     # Get the chunk
#     chunk = tm_facs_data.X[start_idx:end_idx]
    
#     # Convert to dense for the division operation
#     chunk_dense = chunk.toarray()
    
#     # Apply the length normalization
#     chunk_normalized = chunk_dense / gene_len[1].values * scaling_factor
    
#     # Round to integer
#     chunk_rounded = np.rint(chunk_normalized)
    
#     # Convert back to sparse and update the original matrix
#     tm_facs_data.X[start_idx:end_idx] = sparse.csr_matrix(chunk_rounded)
    
#     # Clear memory
#     del chunk_dense, chunk_normalized, chunk_rounded

# # Verify the operation
# assert (tm_facs_data.var.index == gene_len.index).sum() == tm_facs_data.shape[1]
import numpy as np

gene_len = gene_len.reindex(tm_facs_data.var.index).dropna()
tm_facs_data = tm_facs_data[:, gene_len.index]
assert (tm_facs_data.var.index == gene_len.index).sum() == tm_facs_data.shape[1]
tm_facs_data.X = tm_facs_data.X / gene_len[1].values * np.median(gene_len[1].values)
# round to integer
tm_facs_data.X = np.rint(tm_facs_data.X)

In [ ]:
import scanpy as sc
import pandas as pd

tm_adata = tm_droplet_data.concatenate(tm_facs_data)
tm_adata.layers["counts"] = tm_adata.X.copy()
sc.pp.normalize_total(tm_adata, target_sum=1e4)
sc.pp.log1p(tm_adata)
tm_adata.raw = tm_adata  # keep full dimension safe
sc.pp.highly_variable_genes(
    tm_adata,
    flavor="seurat_v3",
    n_top_genes=2000,
    layer="counts",
    batch_key="tech",
    subset=True,
)

In [ ]:
tm_adata

In [10]:
import pickle
# Create the directory if it doesn't exist
import os
os.makedirs(r'./src/data/dann', exist_ok=True)

with open(r'./src/data/dann/all_cell_data.pkl', 'wb') as f:
    pickle.dump(tm_adata, f)